# Enunciado 

Un resorte helicoidal de compresión fabricado, hecho de alambre de piano, tiene un tamaño de alambre de 0.092 in, un diámetro exterior de la espira de 9/16 in, una longitud libre de 4 3/8 in, 21 espiras activas y ambos extremos a escuadra y esmerilados. El resorte está sin granallar, habrá de ensamblarse con una precarga de 5 lb y operará sometido a una carga máxima de 35 lb durante el servicio.

1. Calcule el factor de seguridad que protege contra la falla por fatiga, mediante un lugar geométrico de fatiga por torsión de Gerber y datos de Zimmerli.

2. Repita el inciso *1* usando el lugar geométrico de fatiga por torsión de Sines (la componente del esfuerzo constante no tiene efecto) y los datos de Zimmerli.

3. Repita utilizando el lugar geométrico de falla por torsión de Goodman y datos de Zimmerli

4. Repita utilizando el lugar geométrico de falla de Whal, con datos de Zimmerli.
5. Determine la frecuencia crítica del resorte

### Import necessary libraries

In [1]:
import pandas as pd
from sympy import Eq, solve, Symbol
import numpy as np
from colorama import Fore

### Establecer datos conocidos

In [23]:
d = 0.092 # in
De = 9 / 16  # in
L_f = (4 * 8 + 3) / 8
F_max = 35 #lb
F_min = 5 #lb


### Selección del material

In [5]:
table_10_5 = pd.read_csv('tables/table_10_5.csv')
table_10_5.head()

,Material,Limite tension,limite torsion,Diámetro in,E M-psi,E GPa,G M-psi,G GPa
0,Alambre de piano A228,65-75,45-60,<0.032,29.5,203.4,12.00,82.7
1,Alambre de piano A228,65-75,45-60,0.033-0.063,29.0,200.0,11.85,81.7
2,Alambre de piano A228,65-75,45-60,0.064-0.125,28.5,196.5,11.75,81.0
3,Alambre de piano A228,65-75,45-60,>0.125,28.0,193.0,11.60,80.0
4,Resorte estirado duro A227,60-70,45-55,<0.032,28.8,198.6,11.70,80.7


In [10]:
# Search Cromo-vanadio A231	
count = 2
material  = table_10_5[table_10_5['Material'] == 'Alambre de piano A228'].\
    values[count][1:]
G = material[-2]
porcentaje_sut = int(material[1].split('-')[0])/100
material, porcentaje_sut, G

(array(['65-75', '45-60', '0.064-0.125', 28.5, 196.5, 11.75, 81.0],
       dtype=object),
 0.45,
 11.75)

### Encontrar c

In [21]:
Dm = De - d # in
print(Fore.LIGHTCYAN_EX + 'Dm = ', Dm, 'in', Fore.RESET)

c = Dm/d 
print(Fore.LIGHTGREEN_EX + 'c = ', c, Fore.RESET)


Dm =  0.47050000000000003 in 
c =  5.114130434782609 


### Encontrar el valor de K_b

In [22]:
k_b = (4 * c + 2) / (4 * c - 3)
print(Fore.LIGHTWHITE_EX + 'k_b = ', k_b, Fore.RESET)

k_b =  1.286425902864259 


### Encontrar las fuerzas estándares

In [24]:
F_a = (F_max - F_min) / 2

F_m = (F_max + F_min) /2

print('F_a' + Fore.LIGHTBLUE_EX + ' = ', F_a, 'lb', Fore.RESET)
print('F_m' + Fore.LIGHTBLUE_EX + ' = ', F_m, 'lb', Fore.RESET)

F_a =  15.0 lb 
F_m =  20.0 lb 


### Encontrar los esfuerzos

In [26]:
tau = lambda K_b, F, D_m, d : 8 * K_b * F * D_m / (np.pi * pow(d,3))

tau_a = tau(k_b, F_a, Dm, d)
tau_m = tau(k_b, F_m, Dm, d)

print('tau_a' + Fore.LIGHTBLUE_EX + ' = ', tau_a, 'psi', Fore.RESET)
print('tau_m' + Fore.LIGHTBLUE_EX + ' = ', tau_m, 'psi', Fore.RESET)

tau_a =  29690.1434086965 psi 
tau_m =  39586.857878261995 psi 
